In [ ]:
import numpy as np
import random

In [ ]:
def gen_population(N,sudoku):
  missingNumbers = []
  for i in range(len(sudoku)):
    missingNumbers.append([])
    for j in range(len(sudoku[0])+1):
      if j not in sudoku[i]:
        missingNumbers[i].append(j)
  copy = np.copy(missingNumbers)

  population = []
  for i in range(N):
    population.append([])
    for j in range(len(copy)):
      population[i].append(np.random.permutation(copy[j]))
  
  return np.array(population) 



In [ ]:
def get_fitness(individuo,coordinates,sudoku,maxError):
  errorCount = 0
  sudokuCopy = np.copy(sudoku)
 
  for i in range(len(coordinates)):
    for j in range(len(coordinates[i])):
      sudokuCopy[i][coordinates[i][j]] = individuo[i][j]

  for i in range(len(coordinates)):
    for j in range(len(coordinates[i])):
      #print(coordinates[i][j])
      sudokuCopy[i][coordinates[i][j]] = 0
      iCuadrante = int(i/3) * 3
      jCuadrante = int(coordinates[i][j]/3) * 3
      if individuo[i][j] in sudokuCopy[:,coordinates[i][j]] or individuo[i][j] in sudokuCopy[iCuadrante:iCuadrante+3,jCuadrante:jCuadrante+3]:
        errorCount+=1
      sudokuCopy[i][coordinates[i][j]] = individuo[i][j]
  return (maxError - errorCount) + 1
      

              
        


In [ ]:
def get_coordinates(sudoku):
  coordinates = []
  maxError = 0
  for i in range(len(sudoku)):
    coordinates.append([])
    for j in range(len(sudoku[0])):
      if sudoku[i][j] == 0:
        maxError+=1
        coordinates[i].append(j)
  
  return coordinates,maxError



In [ ]:
def get_elite(fitness,population):
  max = -1
  newElite = np.zeros_like(population[0])
  maxIndex = 0
  for i in range(len(fitness)):
    if fitness[i]>max:
      max = fitness[i]
      maxIndex = i
  return max, population[maxIndex]

In [ ]:
def get_worst(fitness,population,maxFit):
  max = maxFit 
  maxIndex = 0
  for i in range(len(fitness)):
    if fitness[i]<max:
      max = fitness[i]
      maxIndex = i
  return maxIndex

In [ ]:
def get_parents(fitness, population):
  fitSum = np.sum(fitness)
  proportions = fitness/fitSum
  #print(proportions)
  #print(fitness)
  ranges = np.zeros((len(fitness),2))
  startIndex = 0
  for i in range(len(fitness)):
    ranges[i][0] = startIndex
    ranges[i][1] = startIndex + proportions[i]
    startIndex+=proportions[i]
  #print(ranges)

  newPopulation = np.zeros_like((population))
  for i in range(len(fitness)):
    randomN = random.random()
    for j in range(len(fitness)):
      if randomN > ranges[j][0] and randomN < ranges[j][1]:
        newPopulation[i] = np.copy(population[j])
  
  return newPopulation

In [ ]:
def get_crossover(population,N,Pr):
  newPopulation = np.copy(population)
  for i in range(N):
    if random.random() < Pr:
      parent1,parent2 = newPopulation[np.random.randint(N)],newPopulation[np.random.randint(N)]
      cont = 0
      for p1,p2 in zip(parent1,parent2):
        son = np.zeros_like(p1)
        if len(p1) < 4:
          son = p1
        else:
          parentLenght = len(p1)
          corte1 = np.random.randint(1,parentLenght-1)
          corte2 = np.random.randint(corte1,parentLenght)
          son[corte1:corte2] = p1[corte1:corte2]

          for j in range(corte1):
            if p2[j] not in son:
              son[j] = p2[j]
          for j in range(corte2,parentLenght):
            if p2[j] not in son:
              son[j] = p2[j]
          
          for j in range(parentLenght):
            if son[j] == 0:
              for k in range(parentLenght):
                if p2[k] not in son and son[j]==0:
                  son[j] = p2[k]
        newPopulation[i][cont] = son
        cont+=1

  return newPopulation

In [ ]:
def get_mutation(population,Pm):
  newPopulation = np.copy(population)
  N = len(newPopulation)
  for i in range(N):
    for j in range(len(newPopulation[i])):
      if random.random() < Pm:
        random1,random2 = np.random.randint(0,len(newPopulation[i][j])),np.random.randint(0,len(newPopulation[i][j]))
        newPopulation[i][j][random1],newPopulation[i][j][random2] = newPopulation[i][j][random2],newPopulation[i][j][random1]
  return newPopulation


In [ ]:
def gen_algorithm(N,G,Pr,Pm,sudoku):
  Gk = 0
  #print("Initial pop")
  population = gen_population(N,sudoku)
  #print(population)
  coordinates,maxError = get_coordinates(sudoku)
  #print(coordinates,maxError)
  fitness = []
  for i in population:
    fitness.append(get_fitness(i,coordinates,sudoku,maxError))
  elite = get_elite(fitness,population)

  while Gk < G or elite[0] != maxError + 1:
    #print("Parents")
    fitness = []
    for i in population:
      fitness.append(get_fitness(i,coordinates,sudoku,maxError))
    population = np.copy(get_parents(fitness,population))
    #print(population)
  
    #print("Antes de crossover")
    #print(population[:,2])
    population = np.copy(get_crossover(population,N,Pr))
    '''
    print("Crossover")
    print(population[:,2])
    '''
    #print("before elite",get_fitness(elite[1],coordinates,sudoku,maxError))
    
    aux = np.zeros_like(elite[1])
    for i in range(len(aux)):
      aux[i] = np.copy(np.array(elite[1][i]))
    elite = (elite[0],aux)
    
    population = np.copy(get_mutation(population,Pm))
    '''
    print("Mutation")
    print(population[:,2])
    '''
    
    fitness = []
    for i in population:
      fitness.append(get_fitness(i,coordinates,sudoku,maxError))
    
    newElite = get_elite(fitness,population)
    if newElite[0] > elite[0]:
      elite= newElite[0],newElite[1]
    else:
      worst = get_worst(fitness,population,maxError+1)
      best = np.array(elite[1])
      population[worst] = np.copy(best)
    #print("Elite", elite[0],elite[1])
    #print("New Elite",newElite[0],newElite[1])
    #print("Final elite",get_fitness(elite[1],coordinates,sudoku,maxError))
    Gk+=1
    if(Gk%100==0):
      print(elite[0])
  
  return elite

In [ ]:
'''M = [[9,6,0,1,0,4,0,5,8],

 [0,7,8,3,2,5,6,0,9],

 [2,5,0,6,0,0,7,0,1],

 [8,0,1,4,0,7,5,0,6],

 [0,9,6,0,0,2,3,0,7],

 [7,0,5,9,6,1,0,2,4],

 [5,0,0,7,1,0,4,6,2],

 [3,1,7,2,0,6,9,0,0],

 [0,4,0,5,0,8,0,7,3]]

#SUDOKU DE PRUEBA

M = [[2,3,0,0,8,9,0,0,0],
     [1,8,6,0,7,3,0,0,4],
     [4,9,7,2,6,1,0,3,5],
     [7,6,0,0,0,0,0,9,0],
     [9,5,2,6,0,4,0,7,8],
     [0,4,0,0,9,0,1,0,0],
     [0,0,0,0,5,0,2,0,3],
     [8,0,3,1,2,6,0,4,9],
     [0,2,0,3,4,8,6,0,0]]
'''

M = [[5,3,4,6,7,8,9,1,2],

 [6,7,2,1,9,5,3,4,8],

 [1,9,8,3,4,2,5,6,7],

 [8,5,9,7,6,1,4,2,3],

 [4,2,6,8,5,3,7,9,1],

 [7,1,3,9,2,4,8,5,6],

 [9,6,1,5,3,7,2,8,4],

 [2,8,7,4,1,9,6,3,5],

 [0,4,5,2,8,6,1,7,0]]

M = np.array(M)

solution = gen_algorithm(30,1000,0.8,0.3,M) 
print("FITNESS:",solution[0])
print("----FINAL SOLUTION----")
print(solution[1])
coordinates,_ = get_coordinates(M)
mCopy = np.copy(M)
for i in range(len(coordinates)):
  for j in range(len(coordinates[i])):
    mCopy[i][coordinates[i][j]] = solution[1][i][j]

print("----SOLVED SUDOKU----")
printSudoku(mCopy)


/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:792: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, order=order, subok=subok, copy=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


3
3
3
3
3
3
3
3
3
3
FITNESS: 3
----FINAL SOLUTION----
[array([0]) array([0]) array([0]) array([0]) array([0]) array([0])
 array([0]) array([0]) array([3, 9])]
----SOLVED SUDOKU----
5 | 3 | 4 | 6 | 7 | 8 | 9 | 1 | 2 | 
----------------------------------
6 | 7 | 2 | 1 | 9 | 5 | 3 | 4 | 8 | 
----------------------------------
1 | 9 | 8 | 3 | 4 | 2 | 5 | 6 | 7 | 
----------------------------------
8 | 5 | 9 | 7 | 6 | 1 | 4 | 2 | 3 | 
----------------------------------
4 | 2 | 6 | 8 | 5 | 3 | 7 | 9 | 1 | 
----------------------------------
7 | 1 | 3 | 9 | 2 | 4 | 8 | 5 | 6 | 
----------------------------------
9 | 6 | 1 | 5 | 3 | 7 | 2 | 8 | 4 | 
----------------------------------
2 | 8 | 7 | 4 | 1 | 9 | 6 | 3 | 5 | 
----------------------------------
3 | 4 | 5 | 2 | 8 | 6 | 1 | 7 | 9 | 
----------------------------------


In [ ]:
def printSudoku(sudoku):
  for i in range(len(sudoku)):
    for j in range(len(sudoku[i])):
      print(sudoku[i][j],end=" | ")
    print()
    print("----------------------------------")


# Descripción del proyecto

**Representación individual**: **texto en negrita** Los individuos están conformados por los números faltantes de cada fila.

**Selección de padres:** La selección de los padres se implentó por el método de la ruleta, generando los rangos para poder llevar a cabo este proceso

**Crossover:** Decidí utilizar la Partially mapped crossover ya que se estuvieron utilizando permutaciones. Este proceso lo repetía para cada fila, ya que de otra manera se hubieran sobrescrito los tamaños originales de los individuos.

**Mutation:** La mutación fue un simple swap ente dos posiciones aleatorias de una fila.


